In [29]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import fashion_mnist
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


# ***CNN with Keras***

In [30]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0
x_train = x_train[..., None]
x_test = x_test[..., None]


In [31]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [32]:
history = model.fit(x_train, y_train, epochs=10, validation_split=0.1)


Epoch 1/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.7685 - loss: 0.6398 - val_accuracy: 0.8750 - val_loss: 0.3297
Epoch 2/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8869 - loss: 0.3147 - val_accuracy: 0.8972 - val_loss: 0.2806
Epoch 3/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9011 - loss: 0.2691 - val_accuracy: 0.8930 - val_loss: 0.2834
Epoch 4/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9163 - loss: 0.2247 - val_accuracy: 0.9028 - val_loss: 0.2635
Epoch 5/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9258 - loss: 0.1985 - val_accuracy: 0.9117 - val_loss: 0.2461
Epoch 6/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9340 - loss: 0.1754 - val_accuracy: 0.9000 - val_loss: 0.2864
Epoch 7/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9419 - loss: 0.1553 - val_accuracy: 0.9108 - val_loss: 0.2465
Epoch 8/10
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9487 - loss: 0.1387 -

In [33]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test accuracy:", test_acc)


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9133 - loss: 0.2913
Test accuracy: 0.9138000011444092


# ***CNN with Pytorch***

In [34]:
transform = transforms.ToTensor()

train_data = datasets.FashionMNIST(root='./data', train=True,
                                   download=True, transform=transform)
test_data = datasets.FashionMNIST(root='./data', train=False,
                                  download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)



In [35]:
class FashionClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(64 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

In [36]:

model = FashionClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [37]:
def train(model, loader, criterion, optimizer):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for images, labels in loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    accuracy = correct / total
    avg_loss = total_loss / len(loader)
    return avg_loss, accuracy


In [38]:
def evaluate(model, loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            outputs = model(images)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(predicted.numpy())
            all_labels.extend(labels.numpy())

    accuracy = correct / total
    avg_loss = total_loss / len(loader)
    return avg_loss, accuracy, all_preds, all_labels


In [39]:
num_epochs = 10

for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer)
    test_loss, test_acc, _, _ = evaluate(model, test_loader, criterion)
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"Test  Loss: {test_loss:.4f} | Test  Acc: {test_acc:.4f}")
    print('-' * 40)


Epoch 1/10
Train Loss: 0.5312 | Train Acc: 0.8049
Test  Loss: 0.4033 | Test  Acc: 0.8575
----------------------------------------
Epoch 2/10
Train Loss: 0.3447 | Train Acc: 0.8716
Test  Loss: 0.3584 | Test  Acc: 0.8665
----------------------------------------
Epoch 3/10
Train Loss: 0.2948 | Train Acc: 0.8900
Test  Loss: 0.2906 | Test  Acc: 0.8926
----------------------------------------
Epoch 4/10
Train Loss: 0.2611 | Train Acc: 0.9033
Test  Loss: 0.2860 | Test  Acc: 0.8979
----------------------------------------
Epoch 5/10
Train Loss: 0.2365 | Train Acc: 0.9131
Test  Loss: 0.2687 | Test  Acc: 0.9032
----------------------------------------
Epoch 6/10
Train Loss: 0.2132 | Train Acc: 0.9204
Test  Loss: 0.2571 | Test  Acc: 0.9071
----------------------------------------
Epoch 7/10
Train Loss: 0.1962 | Train Acc: 0.9266
Test  Loss: 0.2475 | Test  Acc: 0.9118
----------------------------------------
Epoch 8/10
Train Loss: 0.1767 | Train Acc: 0.9342
Test  Loss: 0.2752 | Test  Acc: 0.9025
-